In [1]:
import pandas as pd
import sqlite3
import io
import re # Biblioteca de Expressões Regulares
from google.colab import files

# --- 1. UPLOAD ---
print("Por favor, faça o upload do arquivo")
uploaded = files.upload()
file_name = next(iter(uploaded))

# Leitura
df_raw = pd.read_csv(io.BytesIO(uploaded[file_name]), dtype={'Ticket': str})

conn = sqlite3.connect('project_db.sqlite')
df_raw.to_sql('raw_titanic', conn, if_exists='replace', index=False)

# --- 2. TRANSFORMAÇÃO (ETL) ---
df = pd.read_sql("SELECT * FROM raw_titanic", conn)

# A) SEPARAÇÃO BÁSICA (Sobrenome, Título, Resto do Nome)
split_comma = df['Name'].str.split(', ', n=1, expand=True)
df['LastName'] = split_comma[0]

split_dot = split_comma[1].str.split('. ', n=1, expand=True)
df['Title'] = split_dot[0]
df['FirstName_Raw'] = split_dot[1] # Nome Bruto para tratamento

# B) PADRONIZAÇÃO DE TÍTULOS
rare_titles = ['Lady', 'Countess','Capt', 'Col', 'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona', 'the Countess']
df['Title'] = df['Title'].replace(rare_titles, 'Rare')
df['Title'] = df['Title'].replace(['Mlle', 'Ms'], 'Miss')
df['Title'] = df['Title'].replace('Mme', 'Mrs')

# C) CORREÇÃO DO NOME FEMININO (Lógica "Mrs")
def clean_firstname(row):
    name = row['FirstName_Raw']
    # Se for mulher casada (Mrs) e tiver parênteses, o nome real está dentro
    if row['Title'] == 'Mrs' and '(' in name:
        match = re.search(r'\((.*?)\)', name) # Regex para pegar texto dentro dos ()
        if match:
            return match.group(1) # Retorna apenas o nome real

    # Para os demais (ou Mrs sem parênteses), removemos apelidos entre aspas/parênteses se houver, ou mantemos original
    # Ex: "Mr. Pastcho ("Pentcho")" -> vira "Pastcho" se quisermos limpar.
    # Vamos manter simples: se não caiu no caso acima, retorna o nome bruto limpo de aspas duplas
    return name.replace('"', '').replace('(', '').replace(')', '')

df['FirstName'] = df.apply(clean_firstname, axis=1)

# D) CRIAÇÃO DA VARIÁVEL IsMarried
# Assumimos que 'Mrs' indica casamento.
df['IsMarried'] = df['Title'].apply(lambda x: 1 if x == 'Mrs' else 0)

# E) DEMAIS TRATAMENTOS (Idade, Deck, Família)
df['Age'] = df['Age'].fillna(df.groupby('Title')['Age'].transform('median')).astype(int)
df['Embarked'] = df['Embarked'].fillna(df['Embarked'].mode()[0])
df['Deck'] = df['Cabin'].astype(str).str[0]
df.loc[df['Cabin'].isnull(), 'Deck'] = 'Unknown'
df.loc[df['Deck'] == 'n', 'Deck'] = 'Unknown'
df['FamilySize'] = df['SibSp'] + df['Parch'] + 1
df['IsAlone'] = (df['FamilySize'] == 1).astype(int)

# --- 3. SELEÇÃO FINAL ---
# Removemos colunas temporárias
cols = ['PassengerId', 'Survived', 'Pclass', 'Title', 'FirstName', 'LastName',
        'IsMarried', 'Sex', 'Age', 'FamilySize', 'IsAlone', 'Ticket', 'Fare', 'Deck', 'Embarked']
df = df[cols]

# Export
df.to_sql('refined_titanic', conn, if_exists='replace', index=False)
df.to_csv('titanic_cleaned.csv', index=False)

from google.colab import files

# Baixar o Banco de Dados (SQLite)
files.download('project_db.sqlite')

# Baixar o CSV Limpo (Opcional, caso queira usar no Power BI depois)
files.download('titanic_cleaned.csv')

print("ETL Refinado Concluído!")
print(df[['Title', 'FirstName', 'LastName', 'IsMarried']].head(10))

Por favor, faça o upload do arquivo


Saving titanic.csv to titanic (4).csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

ETL Refinado Concluído!
    Title                  FirstName   LastName  IsMarried
0      Mr                Owen Harris     Braund          0
1     Mrs     Florence Briggs Thayer    Cumings          1
2    Miss                      Laina  Heikkinen          0
3     Mrs              Lily May Peel   Futrelle          1
4      Mr              William Henry      Allen          0
5      Mr                      James      Moran          0
6      Mr                  Timothy J   McCarthy          0
7  Master              Gosta Leonard    Palsson          0
8     Mrs  Elisabeth Vilhelmina Berg    Johnson          1
9     Mrs                Adele Achem     Nasser          1
